**BERT 모델 불러오기**

In [1]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
# CLS 토큰
cls_token = tokenizer.cls_token
print(f"CLS token: {cls_token}")

# SEP 토큰
sep_token = tokenizer.sep_token
print(f"SEP token: {sep_token}")

# PAD 토큰
pad_token = tokenizer.pad_token
print(f"PAD token: {pad_token}")

# UNK 토큰
unk_token = tokenizer.unk_token
print(f"UNK token: {unk_token}")

# MASK 토큰
mask_token = tokenizer.mask_token
print(f"MASK token: {mask_token}")



CLS token: [CLS]
SEP token: [SEP]
PAD token: [PAD]
UNK token: [UNK]
MASK token: [MASK]


**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 알 수 없는 단어
- [SEP] : 두 문장 구분
- [PAD] : 짧은 문장의 길이를 맞추기 위해 추가 되는 토큰
- [CLS] : 문장, 문서의 시작 부분에 추가.
- [MASK] : 특정 단어를 가리고, 모델이 가려진 단어를 예측할 때 사용

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [3]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : 전체 문장을 대표하는 벡터를 생성하기 위해
- [SEP] 토큰이 추가된 이유 : 한 문장의 끝을 내기위해

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [4]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    ############### EDIT###############
    'sentiment analysis 구현'
      # 입력 문장 토큰화 및 텐서 변환
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)

    # GPU 사용 가능 시 GPU로 이동
    if torch.cuda.is_available():
        inputs = {key: value.cuda() for key, value in inputs.items()}
        model.cuda()

    # 모델 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 소프트맥스 함수를 통해 확률 계산
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # 확률을 리스트로 변환
    probs_list = probs[0].cpu().tolist()

    ###################################

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [17]:
# 매우 부정적인 문장 작성
sentence = "i'm extremely stupid student"

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.18
부정적일 확률: 0.24
중립적일 확률: 0.29
긍정적일 확률: 0.17
매우 긍정적일 확률: 0.11


In [15]:
# 부정적인 문장 작성
sentence = "i'm  stupid student"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.16
부정적일 확률: 0.23
중립적일 확률: 0.29
긍정적일 확률: 0.20
매우 긍정적일 확률: 0.13


In [11]:
# 중립적 문장 작성
sentence = "i'm student"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.12
부정적일 확률: 0.17
중립적일 확률: 0.28
긍정적일 확률: 0.22
매우 긍정적일 확률: 0.20


In [12]:
# 긍정적 문장 작성
sentence = "i'm good student"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.13
부정적일 확률: 0.18
중립적일 확률: 0.29
긍정적일 확률: 0.21
매우 긍정적일 확률: 0.18


In [14]:
# 매우 긍정적인 문장 작성
sentence = "i'm extremely good student"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.15
부정적일 확률: 0.19
중립적일 확률: 0.30
긍정적일 확률: 0.20
매우 긍정적일 확률: 0.16


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 :
단어의 의미, 문장구조, 부사 사용정도, 비유적 표현과 관용구 등